## A CNN to classify EEG data

In [18]:
!pip install pydrive

import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys
import pickle

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus



## Download the dataset

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')

mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)

print("loaded dataset  ...")

o=mat['o']
data=mat['o'].data
labels=mat['o'].marker

newlabels = np.zeros([data.shape[0],4])
found = np.zeros([1,])

#clean the shit out of it
while np.amax(labels) > 3:
  ind = np.argmax(labels)
  found = np.append(found,np.array([ind]), axis=0)
  labels[ind] = 0
  data[ind,:] = np.zeros([22,])
    

print("cleaned data from breaks  ...")

labeled_data = (data,labels)
  

loaded dataset  ...
cleaned data from breaks  ...


In [0]:
print(data.shape)
print(labels.shape)

#adding the label info directly to the data
data = np.hstack((data,labels.reshape((labels.shape[0],1))))


(667000, 22)
(667000,)


##Video Stream Creation

The imported data is converted into a 3D image. The electrodes are reconstructed assuming the head is a 3 dimensional ellipse.
The calculation can be seen at:

https://docs.google.com/spreadsheets/d/1TetDbE-HkAQGnVVIFbPGI4ugbktgdic_TFtsFyWEp0w/edit?usp=sharing

In [0]:
## video maker and data extractor

pos=np.zeros((22,3), dtype=int)

pos[0]=-2,10,0
pos[1]=2,10,0
pos[3]=-4,7,3
pos[4]=4,7,3
pos[5]=-5,0,4
pos[6]=5,0,4
pos[7]=-4,-7,3
pos[8]=4,-7,3
pos[9]=-2,-10,0
pos[10]=2,-10,0
pos[11]=-7,0,-5
pos[12]=7,0,5
pos[13]=-6,6,0
pos[14]=6,6,0
pos[15]=-7,0,0
pos[16]=7,0,0
pos[17]=-6,-6,0
pos[18]=6,-6,0
pos[19]=0,7,4
pos[20]=0,0,6
pos[21]=0,-7,4

def make_3d_point(x,y,z,r,theta,phi):
  return [
      x + int(r * math.cos(theta)*math.sin(phi)),
      y + int(r * math.sin(theta)*math.sin(phi)),
      z + int(r * math.cos(phi))
         ]

def make_3d_data(image3d_data):
  #creating the matrix
  matrix = np.zeros([12,16,22]) # z,x,y
  
  #looping through data
  for point in image3d_data:
    
    # creating the indice by shifting the coordinate system from the center to left corner
    newpoint = [int(point[0]) + 7,int(point[1]) + 10,int(point[2]) + 5]
    
    if newpoint[0] >= matrix.shape[1] or newpoint[1] >= matrix.shape[2] or newpoint[2] >= matrix.shape[0] or newpoint[0] < 0 or newpoint[1] < 0 or newpoint[2] < 0:
      #print("did not add point")
      #print(newpoint)
      #print(matrix.shape)
      continue

    
    #assigning the value
    if abs(matrix[newpoint[2],newpoint[0],newpoint[1]]) <= abs(point[3]):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(point[3])

    
    #creating variables for creating a virtual sphere around poi
    rho = [2,3] # distance (in pixels) that we are interpolating from poi
    #theta  angle in radians    [0,2pi]
    #phi    angle in radians    [0, pi]
    
    #creating new interpolated points
    for r in rho:
      for theta in range(0,360,45):
        for phi in range(0,180,45):
          p = make_3d_point(newpoint[0],newpoint[1],newpoint[2],r,math.radians(theta),math.radians(phi))
          
          #checking if its in bounds
          if p[0] < matrix.shape[1] and p[1] < matrix.shape[2] and p[2] < matrix.shape[0]:
            if abs(matrix[p[2],p[0],p[1]]) <= abs(point[3]/r):
                matrix[p[2],p[0],p[1]] = int(point[3]/r)
                     
  return matrix


def get_data(batch_size, test_batch_size=256):
  

  # Create train and validation splits
  num_samples = len(data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(data, [training_samples, validation_samples])

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=False)
  val_loader = torch.utils.data.DataLoader(validation_data, batch_size, shuffle=False)
  #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
  
  print("loading data")
  
  return train_loader, val_loader #, test_loader



def get_video_data(loader):
  
  #print("get videos:")

  batchsize = loader.shape[0]
  
  training_data   = np.empty([batchsize,22,4])
  
  train_label = np.empty((batchsize))
  
 # print("start collecting training data")
  
  for it,timestep_data in enumerate(loader):
    train3d = np.hstack((pos,timestep_data[0:22].reshape([22,1])))
    training_data[it] = train3d
    train_label[it] = timestep_data[22] 
 
    
  return (training_data, train_label)
  
  
  
def make_video(video_data):
  return make_3d_data(video_data)
  
  
  
def get_training_data(train):

  stream = np.zeros([train[0].shape[0],1,12,16,22])
  
  for i,image in enumerate(train[0]):
    if i >= stream.shape[0]:
      break
    stream[i,0] = make_video(image)
  
  tensor = torch.from_numpy(stream)

  min_v = np.amax(data)

  max_v = np.amin(data)

  limit = max(abs(min_v),abs(max_v))

  tensor = tensor/limit
  
  return (tensor,train[1])

def save_video(batch_size,path):
  
  !mkdir VideoBatches
  
  print("saving video files in VideoBatches with batch size: ",batch_size)
  
  train_loader, val_loader = get_data(batch_size)
  
  print("\nsaving training data\n")
  
  for batch_idx, inputs in enumerate(train_loader):

    if batch_idx <= 1309:
      continue
      
    if batch_idx % 100 == 0:
      print("saving batches: " , batch_idx, "-", batch_idx+100)
      
    result_data = get_video_data(inputs)
    result = get_training_data(result_data)
    
    name = "VideoBatches/train_" + str(batch_idx) + ".pickle"
    
    with open(name, 'wb') as f:
      pickle.dump([result[0], result[1]], f)
      
  print("\nsaving validation data\n")
  
  for batch_idx, inputs in enumerate(train_loader):

    if batch_idx % 100 == 0:
      print("saving batches: " , batch_idx, "-", batch_idx+100)
      
    result_data = get_video_data(inputs)
    result = get_training_data(result_data)
    
    name = "VideoBatches/val_" + str(batch_idx) + ".pickle"
    
    with open(name, 'wb') as f:
      pickle.dump([result[0], result[1]], f)
    
    
    
  
  



In [0]:
#save_video(254,"path")
#! rm -rf VideoBatches/*
#!ls VideoBatches


In [0]:
!pip install git+https://github.com/tsavchyn/coutils.git

In [0]:
from coutils import MyDrive

drive = MyDrive("/drive")
drive.mount()

In [0]:
from coutils import SaveToDrive

# export = SaveToDrive()

# saving the created data to the drive
#for i in range(984,1312+1,1):
 # export.to_drive("VideoBatches/train_"+str(i)+".pickle")
 # export.to_drive("VideoBatches/val_"+str(i)+".pickle")

## CNN

In [0]:
class eeg_CNN(nn.Module):
    
    def __init__(self):
        super(eeg_CNN, self).__init__()
        self.T = 120
        
        self.conv1 = nn.Conv3d(1,5,2, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm1 = nn.BatchNorm3d(5)
        self.conv2 = nn.Conv3d(5,10,(2,3,2), stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm2 = nn.BatchNorm3d(10)
        self.conv3 = nn.Conv3d(10,5,3, stride=1,dilation=1)
        self.batchnorm3 = nn.BatchNorm3d(5)
        self.conv4 = nn.Conv3d(5,5,(3,3,2), stride=1, dilation =1)
        self.batchnorm4 = nn.BatchNorm3d(5)
        self.conv5 = nn.Conv3d(5,5,(1,2,1), stride=1, dilation =1)
        self.batchnorm5 = nn.BatchNorm3d(5)
        self.pooling1 = nn.MaxPool3d((2,2,2), stride=1, dilation=1)
        
        #layer after max pool
        self.conv6 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm6 = nn.BatchNorm3d(5)
        self.conv7 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm7 = nn.BatchNorm3d(5)
        self.conv8 = nn.Conv3d(5,5,(1,2,1), stride=1,dilation=1)
        self.batchnorm8 = nn.BatchNorm3d(5)
        self.pooling2 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        #Layer after next max pool 
        self.conv9 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm9 = nn.BatchNorm3d(5)
        self.conv10 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm10 = nn.BatchNorm3d(5)
        self.conv11 = nn.Conv3d(5,1,(1,2,1), stride=1,dilation=1)
        self.batchnorm11 = nn.BatchNorm3d(5)
        self.pooling3 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        self.fc1 = torch.nn.Linear(in_features=1 * 1 * 16, out_features=4)
      

        
    def forward(self, x):

        # first set of CNNs and then a max pool
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.relu(self.conv3(x)) 
        x = self.batchnorm3(x)
        x = F.relu(self.conv4(x))
        x = self.batchnorm4(x)
        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = self.pooling1(x)
        
        # second set
        
        x = F.relu(self.conv6(x))
        x = self.batchnorm6(x)
        x = F.relu(self.conv7(x))
        x = self.batchnorm7(x)
        x = F.relu(self.conv8(x))
        x = self.batchnorm9(x)
        x = self.pooling2(x)
        
        # set 3  
        
        x = F.relu(self.conv9(x))
        x = self.batchnorm10(x)
        x = F.relu(self.conv10(x))
        x = self.batchnorm11(x)
        x = F.relu(self.conv11(x))
        x = self.pooling3(x)
        
        batch_size, timesteps, C, H, W = x.size()
        
        x = x.view(batch_size * timesteps, C, H, W)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        return F.softmax(x,dim=3)
  
def get_cost_function():
    
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function
    
def get_optimizer(net, lr, wd, momentum):
  #optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=wd, amsgrad=False)
  return optimizer

In [0]:
def test(net, data_type, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  
  with torch.no_grad():
    
    for i in range(1312+1):

      if i % 50 == 0:
        print("on test batch: " , i, "-", i+50)
      
      name = "VideoBatches/" + data_type + "_" + str(i) + ".pickle"
      
      with open(name, 'rb') as f:
        result = pickle.load(f)

      
      t_data   = result[0]
      t_labels = result[1]
      
      # Load data into GPU
      inputs = t_data.float().to(device)
    
      targets = torch.LongTensor(t_labels).to(device)
        
      # Forward pass
      outputs = net(inputs).reshape([inputs.shape[0],4])

      #print(outputs) 
      #print(targets)

      # Apply the loss
      loss = cost_function(outputs, targets)
      #print("loss",loss)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()
      

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net,data_type,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  
  for i in range(1312+1):

    if i % 50 == 0:
      print("on train batch: " , i, "-", i+50)
      
    name = "VideoBatches/" + data_type + "_" + str(i) + ".pickle"
      
    with open(name, 'rb') as f:
      result = pickle.load(f)

      
    t_data   = result[0]
    t_labels = result[1]
      
    # Load data into GPU
    inputs = t_data.float().to(device)
    
    targets = torch.LongTensor(t_labels).to(device)
        
    # Forward pass
    outputs = net(inputs).reshape([inputs.shape[0],4])

    # Apply the loss
    loss = cost_function(outputs, targets)

    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Reset the optimizer
    optimizer.zero_grad()

    # Better print something, no?
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

In [0]:
def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=25):
  
  print("make data loaders")
  
  train_loader = "train"
  val_loader   = "val"
  
  print("network to device")
  net = eeg_CNN().to(device)
  
  print("optimizer")
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  print("cost function")
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy     = test(net, val_loader, cost_function)
  
  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')
  
  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

## Main

In [0]:
main()

make data loaders
network to device
optimizer
cost function
Before training:
on test batch:  0 - 50
on test batch:  50 - 100
on test batch:  100 - 150
on test batch:  150 - 200
on test batch:  200 - 250
on test batch:  250 - 300
on test batch:  300 - 350
on test batch:  350 - 400
on test batch:  400 - 450
on test batch:  450 - 500
on test batch:  500 - 550
on test batch:  550 - 600
on test batch:  600 - 650
on test batch:  650 - 700
on test batch:  700 - 750
on test batch:  800 - 850
on test batch:  850 - 900
on test batch:  900 - 950
on test batch:  950 - 1000
on test batch:  1000 - 1050
on test batch:  1050 - 1100
on test batch:  1100 - 1150
on test batch:  1150 - 1200
on test batch:  1200 - 1250
on test batch:  1250 - 1300
on test batch:  1300 - 1350
on test batch:  0 - 50
on test batch:  50 - 100
on test batch:  100 - 150
on test batch:  150 - 200
on test batch:  200 - 250
on test batch:  250 - 300
on test batch:  300 - 350
on test batch:  350 - 400
on test batch:  400 - 450
on tes